# 

In [15]:
import bluesky
import bluesky.plan_stubs as bps
from apstools.synApps import SscanRecord, SaveData
from ophyd import Device, EpicsSignal, EpicsSignalRO, Component, EpicsMotor
from apstools.plans import run_blocking_function
from ophyd.status import Status
import numpy as np
from epics import caput, caget
import logging
import time
import h5py
import os
import sys
import subprocess
import pathlib
os.environ.pop('QT_QPA_PLATFORM', None)
os.environ.pop('QT_QPA_PLATFORM_PLUGIN_PATH', None)
sys.path.append("/home/beams/STAFF19ID/bluesky/")
# sys.path.append(str(pathlib.Path(__file__).absolute().parent))
from instrument.devices.positionerstream_device import PositionerStream
from instrument.devices.profile_move_device import ProfileMove
from instrument.devices.softgluezynq_device import SoftGlueZynq
from instrument.devices.tetramm_device import TetraMM
from instrument.devices.xspress3_device import Xspress3
from instrument.utils.trajectories import *

/home/beams/STAFF19ID/bluesky/instrument/_iconfig.py


In [16]:
def mkdir(directory):
    if not os.path.exists(directory):
        try:
            os.makedirs(directory)
            print(f"Created directory: {directory}")
        except OSError as e:
            print(f"Failed to create directory: {directory} - {e}")
    else:
        print(f"Directory already exists: {directory}")

In [17]:
def mksubdirs(save_path, subdirs=[]):
    for folder in subdirs:
        path = f"{save_path}{folder}"
        mkdir(path)

In [18]:
def pvget(pv):
    # TODO: refactor to use pvapy package - https://epics.anl.gov/extensions/pvaPy/production/index.html
    try:
        cmd = f'pvget {pv}'
        result = subprocess.getstatusoutput(cmd)
    except subprocess.CalledProcessError as e:
        result = e


In [19]:
def pvput(pv, value):
    # TODO: refactor to use pvapy package - https://epics.anl.gov/extensions/pvaPy/production/index.html
    try:
        cmd = f'pvput {pv} {value}'
        result = subprocess.getstatusoutput(cmd)
    except subprocess.CalledProcessError as e:
        result = e

In [26]:
def create_master_file(basedir, sample_name, groups = ["xspress3", "eiger", "mda", "tetramm", "positions"]):
    with h5py.File(f"{basedir}/{sample_name}_master.h5", "w") as f:
        for group in groups:
            f.create_group(group)
            files = os.listdir(f"{basedir}/{group}")
            if group == "tetramm":
                files = [file for file in files if file.split(".")[-1]=="nc"]
                string_data = [s.encode('utf-8') for s in files]
                dset = f[group].create_dataset("fnames", data=string_data)
                # print(string_data)
            elif group == "mda":
                files = [file for file in files if file.split(".")[-1]=="mda"]
                string_data = [s.encode('utf-8') for s in files]
                dset = f[group].create_dataset("fnames", data=string_data)
                # print(string_data)
            elif group == "xspress3": 
                files = [file for file in files if file.split(".")[-1]=="h5"]
                for file in files:
                    print(f"./{group}/{file}")
                    f[f"/{group}/{file}"] = h5py.ExternalLink(f"/{group}/{file}", "/entry")
            elif group == "positions": 
                files = [file for file in files if file.split(".")[-1]=="h5"]
                for file in files:
                    print(f"./{group}/{file}")
                    f[f"/{group}/{file}"] = h5py.ExternalLink(f"/{group}/{file}", "/stream")

In [31]:
def run_scan(scan_type="fly", trajectory="snake", loop1="2idsft:m1", loop2="2idsft:m2", dwell_time="10", 
             devices={"flyXRF":"XSP3_1Chan:", "tetramm:":"2idsft:TetrAMM1", "profilemove":"2idsft:pm1", "softglue": "2idMZ1:", "positions":"posvr:"},
             sample_name="sample_name", pi_directory = "/mnt/micdata1/save_dev/", comments="",
             l1_center=0, l1_size=0.01, l1_width=0.5, l2_center=0, l2_size=0.01, l2_width=0.5,
             ):
        
    """parse parameters""" 
    if trajectory == "snake":
        x, y, t = snake(dwell_time, l1_size, m1_center, l2_center, l1_width, m2_width)
    elif trajectory == "raster":
        x, y, t = snake(dwell_time, l1_size, m1_center, l2_center, l1_width, m2_width)
    elif trajectory == "spiral":
        pass
    elif trajectory == "lissajous":
        pass
    elif trajectory == "custom":
        pass
        
    x = list(np.round(x, 5))
    y = list(np.round(y, 5))
    npts = len(x)
    frequency= 1/dwell_time/1000
    
    folder_name = sample_name.strip("_")
    save_path = f"{pi_directory}{folder_name}/"
    mkdir(save_path)
    devices= [device for device in devices.keys()]
    subdirs = [detector for detector in devices if detector == ("flyXRF" or "tetramm" or "positions" or "eiger" or "mda" or "xmap")]
    mksubdirs(save_path, subdirs)
    """Set up positioners (move to starting pos)"""
    #TODO: of the parameters in loop1-loop4, figure out which are motors somehow or hardcode them in the devices folder and then import them here. 

    """trigger source"""
    use_softglue_triggers = "softglue" in devices
    
    """setup devices"""
    for device in devices.keys(): 
        if device == "flyXRF":
            if use_softglue_triggers: 
                trigger_mode = 3 #ext trigger
            else: 
                trigger_mode = 1 #internal
            xp3 = Xspress3(devices[device], name="xp3")
            savepath = f"{save_path}{device}"
            yield from xp3.setup_xspress3(xp3, npts, sample_name, savepath, trigger_mode, dwell_time, reset_counter=False)
            
        elif device == "tetramm":
            if use_softglue_triggers: 
                trigger_mode = 1 #ext trigger
            else: 
                trigger_mode = 0 #internal
            tmm = TetraMM(devices[device], name="tmm")
            savepath = f"{save_path}tetramm"
            yield from tmm.setup_tetramm(tmm, npts, sample_name, savepath, trigger_mode, dwell_time, reset_counter=False)
        
        elif device == "profilemove":
            pm1 = ProfileMove(devices[device], name="pm1")
            yield from pm1.setup_profile_move(pm1, x, y, dwell_time)

        elif device == "softglue":
            sgz = SoftGlueZynq(devices[device], name = "sgz")
            yield from sgz.setup_SoftGlueZynq(sgz, npts, frequency)
            
        elif device == "positions": 
            if "xspress3" in devices:
                filenumber = "{:05d}".format(xp3.FileNumber-1)
            else: 
                #TODO: figure out a way to increment softglue filenumber whvery time it closes and a way to reset counter. 
                print("file number not tracked. Not sure how else to set file name if not based on another detector's filenumber")
            postrm = PositionerStream(devices[device], name="postrm")
            filename = f"positions_{filenumber}.h5"
            postrm.setup_positionstream(filename, f"{save_path}positions") 
        else: 
            print(f"unknown device: {device}")
    
    """Start executing scan"""
    st = Status()
    print("Done setting up scan, about to start scan")

    def watch_execute_scan(old_value, value, **kwargs):
        # Watch for scan1.EXSC to change from 1 to 0 (when the scan ends).
        if old_value == (1 or 2 or 3)  and value == 0:
            # mark as finished (successfully).
            if xp3.DetectorState_RBV == 0 or 10: 
                st.set_finished()
                print("finished")
                # Remove the subscription.
                pm1.exsc.clear_sub(watch_execute_scan)

    # TODO need some way to check if devices are ready before proceeding. timeout and exit with a warning if something is missing. 
    # if motors.inpos and pm1.isready and tmm.isready and xp3.isready and sgz.isready and postrm.isready: 
    time.sleep(1)
    ready = True 

    if ready: 
        caput(tmm.Acquire.pvname, 1) #begin acquiring tetramm
        caput(xp3.Acquire.pvname, 1) #begin acquiring xspress3
        pvput(postrm.start_.pvname, 1) #begin position stream
        caput(sgz.send_pulses.pvname, 1) #begin sending pulses
        caput(scan.exsc.pvname, 1) #begin profile move

    pm1.exsc.subscribe(watch_execute_scan) # i think this works
    yield from run_blocking_function(st.wait)

    # #set stop PVs from various devices: 
    # yield from bps.mv(pm1.abort, 1,
    #        tmm.Acquire, 0,
    #        tmm.Capture, 0,
    #        sgz.enbl_dma, 0,
    #        xp3.Capture, 0)
    pvput(postrm.stop_.pvname, 1) #stop position stream
    
    """Set up masterFile"""
    create_master_file(save_path, sample_name, subdirs)

SyntaxError: unterminated string literal (detected at line 12) (3866726772.py, line 12)

In [30]:
###!!!! Instability notes: 
''' if Profile move gets too many points (more than MAX_ARR_Something.. ) the ioc crashes, 
and the tetraMM will need to be power cycled'''

'''Consecutive capture/stop acquire/stop commands seem to cause xspress3 ioc to crash.. need some recovery method. I think it
is from triggering "erase" twice in a row; set erase on start to no. '''

'''xspress3 buffer offloading still painfully slow.. for demo keep aq rate to less than 100Hz'''

'''stop capture sometimes stops working on both xspress3 and tetramm: this was due to micdata file storage instability. '''

'stop capture sometimes stops working on both xspress3 and tetramm: this was due to micdata file storage instability. '

In [ ]:
RE = bluesky.RunEngine()
RE(run_scan(scan_type="fly", trajectory="snake", loop1="2idsft:m1", loop2="2idsft:m2", dwell_time="10", 
             devices={"flyXRF":"XSP3_1Chan:", "tetramm:":"2idsft:TetrAMM1", "profilemove":"2idsft:pm1", "softglue": "2idMZ1:", "positions":"posvr:"},
             sample_name="sample_name", pi_directory = "/mnt/micdata1/save_dev/", comments="",
             l1_center=0, l1_size=0.01, l1_width=0.5, l2_center=0, l2_size=0.01, l2_width=0.5,
             ):